# Tydzień 2.



Do implementacji z zeszłego tygodnia dodać możliwość ułożenia neuronów w topologii siatki sześciokątnej.

Zastosować oba warianty topologii i obie funkcje sąsiedztwa do wektorów danych ze zbiorów

    MNIST
    https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones

(sieć Kohonena uruchomić na zbiorach bez etykiet!).

Przeanalizować otrzymane mapowanie danych uwzględniając etykiety danych. Jak dobrze znalezione klastry odpowiadają podziałowi na klasy?


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import metrics
import math
import matplotlib
import keras
import tensorflow as tf

In [1]:
class kohonen:
    def __init__(self, N,M, dataset, width, architecture):
        self.M=M
        self.N=N
        self.dataset = dataset
        self.neurons = np.random.uniform(np.min(x), np.max(x), size=(N*M,dataset.shape[1]))
        self.width = width
        self.epochs = 0
        if architecture == "hexagon":
            self.dist = self.dist_hex
        else:
            self.dist = self.dist_rec
    
    def learn(self,lambda_, method="gauss"):
        
        self.epochs += lambda_
        
        if method == "mexican":
            neighbor_function = self.mexican_hat
        else:
            neighbor_function = self.gauss
        for iteration in range(lambda_):
            p = np.random.permutation(len(self.dataset))
            x = self.dataset[p]
            for inp in x:
                minimal_pos = np.argmin([np.linalg.norm(neuron-inp) for neuron in self.neurons])
                for i,neuron in enumerate(self.neurons):
                    self.neurons[i] = neuron + neighbor_function(self.dist(self,minimal_pos,i),self.epochs + iteration - lambda_)*self.alpha_fun(self.epochs + iteration - lambda_,self.epochs)*(inp-neuron)
    
    def classify(self):
        return [np.argmin([np.linalg.norm(neuron-inp) for neuron in self.neurons]) for inp in self.dataset]
    
    def limit_neurons(self, n):
        while (len(self.neurons) > n):
            distance = np.array([np.linalg.norm(i-j) for i in self.neurons for j in self.neurons])
            distance = distance[distance!=0]
            distance = distance.reshape(len(self.neurons),len(self.neurons)-1)
            index = np.unravel_index(distance.argmin(), distance.shape)[0]
            self.neurons = np.delete(self.neurons,index,0)

    
    @staticmethod
    def dist_rec(self,x,y):
        a = x%self.N-y%self.N
        b = math.floor(x/self.N)-math.floor(y/self.N)
        
        return self.width*math.sqrt(a**2 + b**2)
    
    @staticmethod
    def dist_hex(self,x,y):
        a = ((x%self.N-y%self.N))
        b = (math.floor(x/self.N)-math.floor(y/self.N)) * math.sqrt(3)/2
        
        if (math.floor(x/self.N)-math.floor(y/self.N))%2==0: 
            a += 1/2
        return self.width*math.sqrt(a**2 + b**2)
    
    @staticmethod
    def alpha_fun(iteration,lambda_):
        return math.exp(-iteration/lambda_)
    
    @staticmethod
    def gauss(x,t):
        return math.exp(-(x*t)**2)
    
    @staticmethod
    def mexican_hat(x,t):
        return abs(math.exp(-(x*t)**2) * (2-4*(x*t)**2))

In [7]:
from scipy.spatial import distance
import numpy as np

def min_interclust_dist(X, label):
    clusters = set(label)
    global_min_dist = np.inf
    for cluster_i in clusters:
        cluster_i_idx = np.where(label == cluster_i)
        for cluster_j in clusters:
            if cluster_i != cluster_j:
                cluster_j_idx = np.where(label == cluster_j)
                interclust_min_dist = np.min(distance.cdist(X[cluster_i_idx], X[cluster_j_idx]))
                global_min_dist = np.min([global_min_dist, interclust_min_dist])
    return global_min_dist

def _inclust_mean_dists(X, label):
    clusters = set(label)
    inclust_dist_list = []
    for cluster_i in clusters:
        cluster_i_idx = np.where(label == cluster_i)
        inclust_dist = np.mean(distance.pdist(X[cluster_i_idx]))
        inclust_dist_list.append(inclust_dist)
    return inclust_dist_list

def mean_inclust_dist(X, label):
    inclust_dist_list = _inclust_mean_dists(X, label)
    return np.mean(inclust_dist_list)

def std_dev_of_inclust_dist(X, label):
    inclust_dist_list = _inclust_mean_dists(X, label)
    return np.std(inclust_dist_list)

def mean_dist_to_center(X, label):
    clusters = set(label)
    inclust_dist_list = []
    for cluster_i in clusters:
        cluster_i_idx = np.where(label == cluster_i)
        cluster_i_mean = np.mean(X[cluster_i_idx], axis=0, keepdims=True)
        inclust_dist = np.mean(distance.cdist(X[cluster_i_idx], cluster_i_mean))
        inclust_dist_list.append(inclust_dist)
    return np.mean(inclust_dist_list)

In [7]:
dataframe = tf.keras.datasets.mnist.load_data(path="mnist.npz")

In [25]:
train_set = dataframe[0][0]
train_labels = dataframe[0][1]

In [28]:
len(train_labels)

60000

In [30]:
train_set[1]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,  51, 159, 253, 159,  50,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  48, 238, 252, 252, 252, 237,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  